# State Of The Art UNET
## JA Engelbrecht

In [1]:
from matplotlib import rc
from jupyterthemes import jtplot
from skimage.util import montage as montage2d
import UNets.Vanilla.UNet_Vanilla as UNet

from MyFunctions.learningRateFunction import LRFinder
from MyFunctions.CreatePaths import CreatePaths
from MyFunctions.LoadImages import LoadImages
from MyFunctions.RunModels import RunModels

from CLR.clr_callback import *

import SimpleITK as sitk
import tensorflow as tf
import pandas as pd
import numpy as np
import importlib
import os

from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split

############ Plot Images/Graphs Functions ############

from matplotlib.colors import LinearSegmentedColormap
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib as mpl

cmap = LinearSegmentedColormap.from_list('mycmap', ['black', 'orange', 'red'])


rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)
mpl.rcParams.update({'font.size': 12})


def set_size(width='thesis', fraction=1, subplots=(1, 1)):
    """Set figure dimensions to avoid scaling in LaTeX.

    Parameters
    ----------
    width: float or string
            Document width in points, or string of predined document type
    fraction: float, optional
            Fraction of the width which you wish the figure to occupy
    subplots: array-like, optional
            The number of rows and columns of subplots.
    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    if width == 'thesis':
        width_pt = 398
    else:
        width_pt = width

    # Width of figure (in pts)
    fig_width_pt = width_pt * fraction
    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    # https://disq.us/p/2940ij3
    golden_ratio = (5**.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio * (subplots[0] / subplots[1])

    return (fig_width_in, fig_height_in)


def showCTImage(IMG, SIZE):
    plt.figure(figsize=(SIZE, SIZE))
    plt.imshow(IMG, alpha=1, cmap='gray')
    plt.axis('off')
    plt.show()


def showCTMontage(IMG, SIZE, SaveFig=False, save_fig_name=""):
    plt.figure(figsize=(SIZE, SIZE))
    plt.imshow(montage2d(IMG), alpha=1, cmap='gray')
    plt.axis('off')
    plt.show()
    if SaveFig:
        save_fig_path = os.path.join(os.curdir, "SavedFigures")
        plt.savefig(os.path.join(save_fig_path,
                                 save_fig_name+".pdf"), bbox_inches='tight')


def showCTMontageOverlay(IMG1, IMG2, SIZE=15, SaveFig=False, save_fig_name=""):
    fig, ax = plt.subplots(figsize=(SIZE, SIZE))
    try:
        ax.imshow(montage2d(IMG1), alpha=1, cmap='gray')
    except:
        print("Error: Img 1")
    try:
        ax.imshow(montage2d(IMG2, fill=0), alpha=0.5,
                  cmap=cmap, interpolation='none')
    except:
        print("Error: Img 2")
    plt.axis('off')

    if SaveFig:
        save_fig_path = os.path.join(os.curdir, "SavedFigures")
        plt.savefig(os.path.join(save_fig_path,
                                 save_fig_name+".pdf"), bbox_inches='tight')
    plt.show()
######################################################

# Set Variables

In [2]:
# Change Class Variable below for different paths e.g. CT/PET
path = CreatePaths(DeviceFlag="PC", ScanTypeFlag="CT", TrainTestFlag="Train")

#DATA_PATH = "D://Masters_Repo//TrainingData//CT_v1"
#IMGS_PATH = path.imgPath()
#MSKS_PATH = path.mskPath()
#OUTPUT_PATH = path.outputPath()

DATA_PATH = "F://MyMasters//Data//TrainingData//PET"
IMGS_PATH = "F://MyMasters//Data//TrainingData//PET//Heart//imgs"
MSKS_PATH = "F://MyMasters//Data//TrainingData//PET//Heart//masks"
OUTPUT_PATH = "F://MyMasters//Output"

ORIENTATION_ENSEMBLE = ["Axial", "Sagittal", "Coronal"]

print("Image Path: "+"\t"+IMGS_PATH+"\n"+"Mask Path: " +
      "\t"+MSKS_PATH+"\n"+"Output Path: "+"\t"+OUTPUT_PATH)

ScanType = "PET"
n_Scans = 25
Orientation = "Coronal"

Image Path: 	F://MyMasters//Data//TrainingData//PET//Heart//imgs
Mask Path: 	F://MyMasters//Data//TrainingData//PET//Heart//masks
Output Path: 	F://MyMasters//Output


# Import and Process Scans

In [3]:
# Note to self! Only change ImgDepth in orientaitons other than axial. Otherwise interpolating unnecessarily!!
PET_Images = LoadImages(ScanType=ScanType, ScanClass="Image",
                       ImgPath=IMGS_PATH, n_Scans=n_Scans, ImgSize=256, ImgDepth=256, Orientation=Orientation).LoadScans()
PET_Masks = LoadImages(ScanType=ScanType, ScanClass="Mask",
                      MskPath=MSKS_PATH, n_Scans=n_Scans, ImgSize=256, ImgDepth=256, Orientation=Orientation).LoadScans()

########################## Split Into Train and Test Set ##########################
X, X_Val, y, y_Val = train_test_split(
    PET_Images, PET_Masks, test_size=0.15, random_state=42)

del PET_Images, PET_Masks

y = tf.cast(y, dtype='float32')
y_Val = tf.cast(y_Val, dtype='float32')

Reading the following PET Images:
CB_001_PET_M0.nii.gz
CB_002_PET_M0.nii.gz
CB_003_PET_M0.nii.gz
CB_004_PET_M1.nii.gz
CB_005_PET_M1.nii.gz
CB_007_PET_M0.nii.gz
CB_009_PET_M0.nii.gz
CB_013_PET_M0.nii.gz
CB_020_PET_M0.nii.gz
CB_022_PET_M0.nii.gz
CB_024_PET_M1.nii.gz
CB_029_PET_M1.nii.gz
CB_034_PET_M0.nii.gz
CB_077_PET_M1.nii.gz
CB_080_PET_M1.nii.gz
CB_086_PET_M0.nii.gz
CB_087_PET_M0.nii.gz
CB_088_PET_M1.nii.gz
CB_089_PET_M1.nii.gz
CB_090_PET_M1.nii.gz
CB_100_PET_M1.nii.gz
CB_101_PET_M0.nii.gz
CB_102_PET_M1.nii.gz
CB_105_PET_M1.nii.gz
CB_108_PET_M1.nii.gz
Reading the following PET Masks:
CB_001_PET_Heart_M0.nii.gz
CB_002_PET_Heart_M0.nii.gz
CB_003_PET_Heart_M0.nii.gz
CB_004_PET_Heart_M1.nii.gz
CB_005_PET_Heart_M1.nii.gz
CB_007_PET_Heart_M0.nii.gz
CB_009_PET_Heart_M0.nii.gz
CB_013_PET_Heart_M0.nii.gz
CB_020_PET_Heart_M0.nii.gz
CB_022_PET_Heart_M0.nii.gz
CB_024_PET_Heart_M1.nii.gz
CB_029_PET_Heart_M1.nii.gz
CB_034_PET_Heart_M0.nii.gz
CB_077_PET_Heart_M1.nii.gz
CB_080_PET_Heart_M1.nii.gz
CB_

## Expand Arrays with a 4'th Singular Dimension (Grayscale Images)

In [4]:
X = np.expand_dims(X, axis=3)
y = np.expand_dims(y, axis=3)
X_Val = np.expand_dims(X_Val, axis=3)
y_Val = np.expand_dims(y_Val, axis=3)

## Create Data Augmentation Generator

In [5]:
dataAug = dict(rotation_range=15,
               zoom_range=0.15,
               horizontal_flip=True,
               vertical_flip=True)

image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**dataAug)
mask_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**dataAug)
seed = 42

image_datagen.fit(X, augment=True, seed=seed)
mask_datagen.fit(y, augment=True, seed=seed)

## View Augmented Scans Overlayed with Masks

In [ ]:
X_Aug = image_datagen.flow(X, batch_size=1, seed=seed)
y_Aug = mask_datagen.flow(y, batch_size=1, seed=seed)
viewImages = np.zeros((200, 256, 256, 1))
viewMasks = np.zeros((200, 256, 256, 1))
for i in range(199):
    viewImages[i, :, :, :] = X_Aug.next()[0]
    viewMasks[i, :, :, :] = y_Aug.next()[0]

In [ ]:
showCTMontageOverlay(IMG1=viewImages[0:199, :, :, 0], IMG2=viewMasks[0:199, :, :, 0],
                     SIZE=25, SaveFig=False, save_fig_name="")

# U-Net

## Preparing to Create U-Net

In [6]:
############## Functions to Log Training of U-Net ##############
def get_run_logdir(root_logdir, input_string):
    import time
    if not input_string:
        run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    else:
        run_id = os.path.join(
            input_string, time.strftime("run_%Y_%m_%d-%H_%M_%S"))
    return os.path.join(root_logdir, run_id)


def create_logdir(modelName):
    root_logdir = os.path.join(os.curdir, "My_logs")
    run_logdir = get_run_logdir(root_logdir, modelName)
    return run_logdir
################################################################

########## Custom Loss Function for Dice Coeffiecient ##########
# https://towardsdatascience.com/dealing-with-class-imbalanced-image-datasets-1cbd17de76b5


def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)


def tversky(y_true, y_pred, smooth=1, alpha=0.7):
    y_true_pos = tf.keras.backend.flatten(y_true)
    y_pred_pos = tf.keras.backend.flatten(y_pred)
    true_pos = tf.keras.backend.sum(y_true_pos * y_pred_pos)
    false_neg = tf.keras.backend.sum(y_true_pos * (1 - y_pred_pos))
    false_pos = tf.keras.backend.sum((1 - y_true_pos) * y_pred_pos)
    return (true_pos + smooth) / (true_pos + alpha * false_neg + (1 - alpha) * false_pos + smooth)


def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true, y_pred)


def focal_tversky_loss(y_true, y_pred, gamma=4/3):
    tv = tversky(y_true, y_pred)
    return tf.keras.backend.pow((1 - tv), gamma)
################################################################

## Model Admin..

In [7]:
MyModelName = 'U-Net_PET_Vanilla_Heart_'+ Orientation
MyLogdir = create_logdir(MyModelName)
MyModelSaveRoot = os.path.join(os.curdir, "TrainedModels")
MyModelSavePath = os.path.join(MyModelSaveRoot, MyModelName+".h5")

print(MyLogdir)
print(MyModelSavePath)
print(MyModelName)

.\My_logs\U-Net_PET_Vanilla_Heart_Coronal\run_2020_10_26-08_35_16
.\TrainedModels\U-Net_PET_Vanilla_Heart_Coronal.h5
U-Net_PET_Vanilla_Heart_Coronal


## Create UNet

In [ ]:
del MyModel
# importlib.reload(UNet)

In [8]:
MyModel = UNet.UNet_Vanilla(input_shape=(256, 256, 1)).CreateUnet()
MyModel.compile(optimizer=Adam(learning_rate=1e-4),
                loss=dice_coef_loss, metrics=[tf.keras.metrics.MeanIoU(num_classes=2)])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 64) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 64) 448         activation[0][0]                 
______________________________________________________________________________________________

## Some Callbacks

In [9]:
csv_logger_cb = tf.keras.callbacks.CSVLogger(
    os.path.join(MyModelSaveRoot, MyModelName+".csv"), append=True)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(MyModelSavePath,
                                                   monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=15, restore_best_weights=True, monitor='val_loss')
tensorboard_cb = tf.keras.callbacks.TensorBoard(MyLogdir)

## Find the Proper Range for the Learning Rate

In [10]:
batch_size = 8
steps_p_epoch = np.ceil(1000/batch_size)

In [ ]:
# Use small subset of data
image_generator = image_datagen.flow(
    X[0:1000, :, :, :], batch_size=batch_size, seed=seed)
mask_generator = mask_datagen.flow(
    y[0:1000, :, :, :], batch_size=batch_size, seed=seed)
train_generator = zip(image_generator, mask_generator)

In [ ]:
lr_finder = LRFinder(min_lr=1e-5, max_lr=1e-2,
                     steps_per_epoch=steps_p_epoch, epochs=3)
MyModel.fit(train_generator, steps_per_epoch=steps_p_epoch,
            epochs=3, verbose=1, callbacks=[lr_finder])
lr_finder.plot_loss()

In [11]:
#learningRates = "0.001-0.006"
# 1st 0.0005-0.006
# 2nd 0.00001-0.0005

base_lr = 4e-4
max_lr = 1e-3

clr_triangular_cb = CyclicLR(
    base_lr=base_lr, max_lr=max_lr, mode='triangular2', step_size=5*X.shape[0])

### Document Compile Parameters

In [12]:
#Optimizer = "SGD, Momentum = 0.9, Nestrov = True"
Optimizer = "Adam"
loss = "Dice_Loss"

### Set Training Patameters

In [14]:
steps_p_epoch = np.ceil(X.shape[0]/batch_size)
image_generator = image_datagen.flow(X, batch_size=batch_size, seed=seed)
mask_generator = mask_datagen.flow(y, batch_size=batch_size, seed=seed)
train_generator = zip(image_generator, mask_generator)
epochs = 20

## Train U-Net

### Initialise Training

In [19]:
MyModel.fit(train_generator, steps_per_epoch=steps_p_epoch, epochs=epochs, verbose=1, validation_data=(X_Val, y_Val),
            callbacks=[checkpoint_cb, early_stopping_cb, clr_triangular_cb, tensorboard_cb, csv_logger_cb])

  ...
    to  
  ['...']
Train for 680.0 steps, validate on 960 samples
Epoch 1/20
679/680 [============================>.] - ETA: 0s - loss: 0.5450 - mean_io_u: 0.6617
Epoch 00001: val_loss did not improve from 0.47488
680/680 [==============================] - 566s 833ms/step - loss: 0.5452 - mean_io_u: 0.6616 - val_loss: 0.5936 - val_mean_io_u: 0.6468
Epoch 2/20
679/680 [============================>.] - ETA: 0s - loss: 0.5191 - mean_io_u: 0.6662
Epoch 00002: val_loss did not improve from 0.47488
680/680 [==============================] - 582s 856ms/step - loss: 0.5194 - mean_io_u: 0.6659 - val_loss: 0.5651 - val_mean_io_u: 0.6590
Epoch 3/20
679/680 [============================>.] - ETA: 0s - loss: 0.5063 - mean_io_u: 0.6629
Epoch 00003: val_loss did not improve from 0.47488
680/680 [==============================] - 612s 899ms/step - loss: 0.5070 - mean_io_u: 0.6628 - val_loss: 0.5469 - val_mean_io_u: 0.6253
Epoch 4/20
679/680 [============================>.] - ETA: 0s - loss: 0.5

In [20]:
#MyModel.load_weights(MyModelSavePath)
clr_triangular_cb._reset()
#base_lr = 1e-4
#max_lr = 6e-4
#clr_triangular_cb = CyclicLR(
#    base_lr=base_lr, max_lr=max_lr, mode='triangular2', step_size=5*X.shape[0])

In [21]:
MyModel.fit(train_generator, steps_per_epoch=steps_p_epoch, epochs=epochs, verbose=1, validation_data=(X_Val, y_Val),
            callbacks=[checkpoint_cb, early_stopping_cb, clr_triangular_cb, tensorboard_cb, csv_logger_cb])

  ...
    to  
  ['...']
Train for 680.0 steps, validate on 960 samples
Epoch 1/20
679/680 [============================>.] - ETA: 0s - loss: 0.4287 - mean_io_u: 0.6887
Epoch 00001: val_loss did not improve from 0.44675
680/680 [==============================] - 594s 873ms/step - loss: 0.4280 - mean_io_u: 0.6887 - val_loss: 0.4900 - val_mean_io_u: 0.6336
Epoch 2/20
679/680 [============================>.] - ETA: 0s - loss: 0.4642 - mean_io_u: 0.6732
Epoch 00002: val_loss did not improve from 0.44675
680/680 [==============================] - 627s 923ms/step - loss: 0.4650 - mean_io_u: 0.6732 - val_loss: 0.4561 - val_mean_io_u: 0.6363
Epoch 3/20
679/680 [============================>.] - ETA: 0s - loss: 0.3989 - mean_io_u: 0.7042
Epoch 00003: val_loss improved from 0.44675 to 0.41901, saving model to .\TrainedModels\U-Net_PET_Vanilla_Heart_Coronal.h5
680/680 [==============================] - 619s 910ms/step - loss: 0.3983 - mean_io_u: 0.7042 - val_loss: 0.4190 - val_mean_io_u: 0.6776
E

KeyError: 'val_loss'

In [ ]:
MyModel.fit(X, y, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_Val, y_Val),
            callbacks=[checkpoint_cb, early_stopping_cb, clr_triangular_cb, tensorboard_cb, csv_logger_cb])

In [ ]:
clr_triangular_cb._reset()

## Load U-Net

In [18]:
MyModel.load_weights(MyModelSavePath)

# Write Model Parameters to Text File

In [ ]:
MyModelParameters_Strings = ["ScanType", "n_Scans",
                             "Orientation", "Optimizer", "Loss", "batch_size", "epochs"]
MyModelParameters_values = [ScanType, n_Scans,
                            Orientation, Optimizer, loss, batch_size, 25]

TextFileName = MyModelName+".txt"
TextFilePath = os.path.join(os.curdir, "TrainedModels", TextFileName)

with open(TextFilePath, "w") as file:
    file.write("Parameters for " + MyModelName + ":\n\n")
    for parameter in enumerate(MyModelParameters_Strings):
        file.write(parameter[1] + ": " +
                   str(MyModelParameters_values[parameter[0]])+"\n")
    file.close()

# Check Performance on Test Set
## View Predicted Images Over Masks

In [ ]:
try:
    y_predict = MyModel.predict(X_Val, batch_size=10, verbose=1)
except:
    X_Val = np.squeeze(X_Val)
    print("Error: Input to Model has to be 4D (x, y, x, 1)")
    print("Reshaping..")
    X_Val = np.expand_dims(X_Val, axis=3)
    y_predict = MyModel.predict(X_Val, batch_size=10, verbose=1)

In [ ]:
y_predict = np.squeeze(y_predict)
X_Val = np.squeeze(X_Val)

try:
    X_Val = np.squeeze(X_Val)
except:
    pass
try:
    y_threshold = np.squeeze(y_threshold)
except:
    pass
try:
    y_Val = np.squeeze(y_Val)
except:
    pass

y_new = np.ma.masked_where(y_predict > 0, y_predict, copy=False)

showCTMontageOverlay(IMG1=X_Val[0:250, :, :],
                     IMG2=y_predict[0:250, :, :], SIZE=25, SaveFig=False, save_fig_name="Predicted Masks on Actual Masks")

# Check Performance on Test Image

In [ ]:
del LoadImages

In [ ]:
ScanName = "CB_130_PET_M0"
TestImage, Orig_Size, MetaData = LoadImages(ScanType="PET", ScanClass="Image", ScanName=ScanName+".nii.gz",
                                            ImgPath="F:\\MyMasters\\Data\\TestingData\\PET\\imgs", Orientation=Orientation).LoadScan()
RunModels(OutPath=OUTPUT_PATH, ScanName="P"+ScanName, Scan=TestImage, Scan_Size=Orig_Size,
          Scan_Metadata=MetaData, Model=MyModel, Orientation=Orientation).runModel()

In [ ]:
Orig_Size